In [ ]:
# 4.0
# Model I/O : 언어 모델과의 인터페이스(promt+language model+output parser)
# retrieval : 외부데이터를 모델에 제공하는 방법
# chain : modelIO랑 뭐가 다름
# agent : chain이 필요한 도구들 제공 
# memory : 
# callback : 요청에 대해 모델이 어떤작업을 하는지 체크

In [ ]:
# 4.1 fewShotPromptTemplate
# prompt template를 사용하는이유 : 디스크에 저장하고 load하는 일련의 과정을 알아서 처리해줌
# FewShot : llm이 어떻게 대답해야 할 지를 예시(형식화)를 줌
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [  
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt, # 어떻게 예제들을형식화할지
    examples=examples,
    suffix="Human: What do you know about {country}?", # 질문을 포함시킴
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Turkey"})

In [ ]:
# 4.2 fewShotChatEmssagePromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# AI에 answer을 넣어주면서 llm에 이런 히스토리가 있다 라고 속이는 원리
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})

In [ ]:
# 4.3 LengthBasedExampleSelector
# example이 많을때 유동적으로 선택하는 내용,비용문제
# maxLength로 자를거면 그냥 example수를 제한하면되잖아
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

In [ ]:
# 4.4 Serializtion and Composition
# Serializtion : 불러오기/저장
# Composition : promptTemplate를 이용해 결합하는

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

# from langchain.prompts import load_prompt
# prompt=load_prompt("./prompt.json")
# prompt=load_prompt("./prompt.yaml")
# prompt.format(country="Germany")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [ ]:
# 4.5 Caching
# 캐싱을 통해 llm의 답변을저장 할 수 있음.동일질문에 대해 재사용
# 

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache()) # memory cache
set_llm_cache(SQLiteCache("cache.db")) # DB에 캐싱
# 캐싱에 관련된로그를 띄워줌/ prompt가 뭔지, 모델이름등 을 보여줌/ chain에서 유용한기능
# set_debug(True) 

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")


In [ ]:
# 4.6 Serializaion

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)


from langchain.llms.openai import OpenAI
chat= OpenAI(temperature=0.1,max_tokens=450,model="gpt-3.5-turbo-16k")
chat.save('model.json')

from langchain.loading import load_llm
chat =load_llm('model.json')

In [ ]:
# 5.0 ConversationBufferMemory
# memory는 5종류가 있음,API는 모두 동일
# ConversationBufferMemory : 대화가 길어질수록 메모리가 증가하는 비효율, 제일 간단함,중복도 허용하는거 같은데
# text completion, 예측,자동완성등에 유용

from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True) 
#chatModel을 사용하고 싶은경우에 return_messages=True하면 return을 chatmodel로 함


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

inputs = {"message": "hi im bob"}
response = chain.invoke(inputs)
response

In [ ]:
# 5.1 ConversationBufferWindowMemory
# 대화의특정 부분만을저장,선입선출형태,k 파라미터로 대화저장갯수 지정
# 최근 대화만 반영하여 답하는 단점

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)


In [ ]:
# 5.2 ConversationSummaryMemory
# 메세지를 그대로 저장하는게 아니라 대화를 요약해서 저장
# 긴대화의 경우 유용,초반에는 다른 메모리저장보다 더많은 용량사용(기능설명등의 내용으로 초반에는 많은 메모리 필요)

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 메모리를 실행하는데도 돈이 든다고함
memory = ConversationSummaryMemory(llm=llm)



def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Kddorea is so pretty", "I wish I could go!!!")
get_history()

In [ ]:
# 5.3 ConversationSummaryBufferMemory
# Summary + buffer 메모리 조합
# 메모리에 보내온 메세지수를 저장,limit에 도착하면 오래된 메세지들은 요약함
# 메세지의 토큰을 기준으로하고, system message에 요약하네

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
get_history()
# {'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
#   AIMessage(content='Wow that is so cool!')]}
add_message("South Korea is so pretty", "I wish I could go!!!")
get_history()
# {'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
#   AIMessage(content='Wow that is so cool!'),
#   HumanMessage(content='South Korea is so pretty'),
#   AIMessage(content='I wish I could go!!!')]}
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
get_history()
# {'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
#   AIMessage(content='Wow that is so cool!'),
#   HumanMessage(content='South Korea is so pretty'),
#   AIMessage(content='I wish I could go!!!'),
#   HumanMessage(content='How far is Korea from Argentina?'),
#   AIMessage(content="I don't know! Super far!")]}
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")
get_history()

In [ ]:
# 5.4 ConversationKGMemory 
# knowledge graph를 그린다는데, 중요한 내용들만 뽑아 요약
# knowledge graph에서 히스토리가 아닌 엔티티를 가져옴
# 

from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is Nicolas"})
add_message("Nicolas likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "what does nicolas like"})
# {'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.')]}

In [ ]:
# 5.5 Memory on LLMChain
# 우리의 memory를 chain에 전달하는 내용(memory에만 저장되고 llm에 전달안되면 의미가 없음)
# llm chain is off-the-shelf chain, that means general purpose chain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history", 
)

template = """
    You are a helpful AI talking to a human.

    {chat_history} #####여기가 memory삽입부
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True, # prompt log를 확인 가능
)

chain.predict(question="My name is Nico")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")

In [ ]:
# 5.6 Chat Based Memory 
# 누가 보냈는지 모르거나,메세지 양등이 예측하기 어려울떄 Placeholder 통해 대체
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")

chain.predict(question="I live in Seoul")

chain.predict(question="What is my name?")

In [ ]:
# 5.7 LCEL Based Memory 
# 체인을 만드는데 메모리랑 프롬프트가 안바뀐데
# invoke시 chat_history를 계속 주입해줘야하는데 RunnablePassthrough를 통한 체인으로 해결
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
invoke_chain("My name is nico")
invoke_chain("What is my name?")

In [ ]:
# 5.8 Recap
# prompt에 기록들을 넣는것은 개발자몫